In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import pathlib
import os

import plotly.graph_objects as go
import ipywidgets as widgets

In [2]:
DATA_PATH = "../data"

In [3]:
files = os.listdir(DATA_PATH)
files = list(filter(lambda x: x.endswith(".pkl"), files))
files = list(filter(lambda x: x.startswith("henon_tunes"), files))

# get numbers from files
numbers = list(map(lambda x: float((x.split("_")[-1])[:-4]), files))

# make a dictionary
d = dict(zip(numbers, [os.path.join(DATA_PATH, f) for f in files]))

In [4]:
epsilon = 0.0

In [5]:
with open(d[epsilon], "rb") as f:
    data = pickle.load(f)

In [6]:
data

{'settings': {'min_power_of_two': 5,
  'max_power_of_two': 14,
  'samples': 500,
  'x_extents': [0.0, 1.0],
  'y_extents': [0.0, 1.0],
  'omega_x': 0.168,
  'omega_y': 0.201,
  'x_flat': array([0.        , 0.00200401, 0.00400802, ..., 0.99599198, 0.99799599,
         1.        ]),
  'y_flat': array([0., 0., 0., ..., 1., 1., 1.])},
 'birkhoff_tunes':      from     to                                             tune_x  \
 0       0     32  [1.0, 0.16799904805791022, 0.16799767750313643...   
 1      32     64  [1.0, 0.16800189492532636, 0.16800337727547188...   
 2       0     64  [1.0, 0.16800196322004435, 0.16800349672996584...   
 3      64    128  [1.0, 0.16799429596350912, 0.1679881847099518,...   
 4       0    128  [1.0, 0.16799954640766124, 0.1679986807783559,...   
 5     128    256  [1.0, 0.16800001115540597, 0.16799961652023132...   
 6       0    256  [1.0, 0.1680009849037054, 0.1680015581030544, ...   
 7     256    512  [1.0, 0.16800101820181434, 0.16800162369513505...   
 

In [60]:
def format_data(data, power, element):
    settings = data["settings"]
    samples = settings["samples"]
    if element == "birkhoff":
        dataframe = data["birkhoff_tunes"]
    elif element == "fft":
        dataframe = data["fft_tunes"]
    else:
        raise ValueError("element must be either 'birkhoff' or 'fft'")

    power_middle = 2**power
    power_to = 2**(power+1)

    dataframe_middle_x = dataframe[(dataframe["from"] == 0) & (dataframe["to"] == power_middle)]["tune_x"].array[0].reshape(samples, samples)
    dataframe_middle_y = dataframe[(dataframe["from"] == 0) & (dataframe["to"] == power_middle)]["tune_y"].array[0].reshape(samples, samples)

    dataframe_to_x = dataframe[(dataframe["from"] == power_middle) & (dataframe["to"] == power_to)]["tune_x"].array[0].reshape(samples, samples)
    dataframe_to_y = dataframe[(dataframe["from"] == power_middle) & (dataframe["to"] == power_to)]["tune_y"].array[0].reshape(samples, samples)

    diff_x = dataframe_to_x - dataframe_middle_x
    diff_y = dataframe_to_y - dataframe_middle_y

    diff = np.sqrt(diff_x**2 + diff_y**2)
    
    diff = np.log10(diff)
    diff[np.isnan(diff)] = np.nanmax(diff)
    diff[np.isinf(diff)] = np.nanmax(diff)
    #diff[np.isnan(diff)] = -1.0
    return diff

In [61]:
format_data(data, 6, "birkhoff")

/tmp/ipykernel_124973/3474184364.py:25: RuntimeWarning:

divide by zero encountered in log10



array([[-0.62444423, -5.11536001, -4.81496751, ..., -0.62444423,
        -0.62444423, -0.62444423],
       [-1.27630489, -4.84509188, -4.51075604, ..., -0.62444423,
        -0.62444423, -0.62444423],
       [-1.27629344, -4.59624186, -4.54373981, ..., -0.62444423,
        -0.62444423, -0.62444423],
       ...,
       [-0.62444423, -0.62444423, -0.62444423, ..., -0.62444423,
        -0.62444423, -0.62444423],
       [-0.62444423, -0.62444423, -0.62444423, ..., -0.62444423,
        -0.62444423, -0.62444423],
       [-0.62444423, -0.62444423, -0.62444423, ..., -0.62444423,
        -0.62444423, -0.62444423]])

In [62]:
dataframe = data["fft_tunes"]

In [63]:
type(dataframe[(dataframe["from"] == 0) & (dataframe["to"] == 2**7)]["tune_x"].array[0][-1])

numpy.float64

In [64]:
format_data(data, 7, "birkhoff")

/tmp/ipykernel_124973/3474184364.py:25: RuntimeWarning:

divide by zero encountered in log10



array([[-0.68689325, -6.33278271, -6.02884393, ..., -0.68689325,
        -0.68689325, -0.68689325],
       [-1.73635183, -5.74512688, -5.45255272, ..., -0.68689325,
        -0.68689325, -0.68689325],
       [-1.73611937, -5.45777867, -5.44647154, ..., -0.68689325,
        -0.68689325, -0.68689325],
       ...,
       [-0.68689325, -0.68689325, -0.68689325, ..., -0.68689325,
        -0.68689325, -0.68689325],
       [-0.68689325, -0.68689325, -0.68689325, ..., -0.68689325,
        -0.68689325, -0.68689325],
       [-0.68689325, -0.68689325, -0.68689325, ..., -0.68689325,
        -0.68689325, -0.68689325]])

In [80]:
power_of_two = widgets.IntSlider(
    value=7,
    min=data["settings"]["min_power_of_two"],
    max=data["settings"]["max_power_of_two"],
    step=1,
    description="Power of 2:",
    continuous_update=False,
)

text_power_of_two = widgets.IntText(
    value= 2 ** power_of_two.value,
    description="N turns: "
)

d_keys = list(sorted(d.keys()))

epsilon_dropdown = widgets.Dropdown(
    options=d_keys,
    value=epsilon,
    description="Epsilon:",
    disabled=False,
)

container = widgets.HBox(
    [power_of_two, text_power_of_two, epsilon_dropdown],
)

# create heatmap with plotly
w1 = go.FigureWidget(data=go.Heatmap(
    z=format_data(data, power_of_two.value, "fft"), colorscale="Viridis"),
    skip_invalid=True,
)
w1.update_layout(
    title_text="FFT tune, epsilon = {}".format(epsilon_dropdown.value),
    xaxis_title="x", yaxis_title="y"
)
w2 = go.FigureWidget(data=go.Heatmap(
    z=format_data(data, power_of_two.value, "birkhoff"), colorscale="Viridis"),
    skip_invalid=True,
)
w2.update_layout(
    title_text="Birkhoff tune, epsilon = {}".format(epsilon_dropdown.value),
    xaxis_title="x", yaxis_title="y"
)
img_container = widgets.HBox([w1, w2])


/tmp/ipykernel_124973/3474184364.py:25: RuntimeWarning:

divide by zero encountered in log10



In [78]:
def response(change):
    epsilon = epsilon_dropdown.value
    power = power_of_two.value
    text_power_of_two.value = 2 ** power
    with open(d[epsilon], "rb") as f:
        data = pickle.load(f)
    w1.data[0].z = format_data(data, power, "fft")
    w2.data[0].z = format_data(data, power, "birkhoff")
    w2.update_layout(title_text="FFT Tunes, epsilon = {}".format(epsilon))
    w1.update_layout(title_text="Birkhoff Tunes, epsilon = {}".format(epsilon))

power_of_two.observe(response, names="value")
epsilon_dropdown.observe(response, names="value")


In [79]:
widgets.VBox([container, img_container])